In [1]:
import pandas as pd
from datetime import datetime, timedelta  
from py2neo import Graph, Node, Relationship

In [2]:
data = pd.read_excel(open('Report form for TU (การตอบกลับ).xlsx', 'rb'), sheet_name='การตอบแบบฟอร์ม 1')  

In [3]:
data.columns

Index(['ประทับเวลา', 'คำนำหน้าชื่อ', 'ชื่อ-นามสกุล (Full name)', 'เพศ (Sex)',
       'อายุ (Age)', 'เบอร์โทรศัพท์ (Telephone no.)', 'Email',
       'ที่อยู่ปัจจุบัน (โปรดระบุ) ', 'ที่อยู่ปัจจุบันโปรดระบุ(Address)',
       'อาชีพ (Occupation)', 'คณะ/สังกัด(Faculty/Department)',
       'ศูนย์ (Campus)',
       'อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)',
       'เดินทางเข้าประเทศเมื่อวันที่ (Entered Thailand on) หากไม่ได้มีประวัติการเดินทางให้ลงวันที่สัมผัสใกล้ชิดผู้เข้าข่ายหรือยืนยันโรคติดเชื้อ',
       'โดยสายการบิน (Airline)', 'เที่ยวบินที่ (Fight no.) ',
       'เลขที่นั่ง (Seat no.)', 'อาการไข้ Fever',
       'อาการอื่นๆ (Sign and symptoms)',
       'วันที่เริ่มป่วย(Date of illness onset)',
       'วันที่เข้ารับการรักษาครั้งเเรก (Date of first visit/hospitalization)',
       'ชื่อสถานบริการ (Name of health facility of first visit/hospitalization)',
       'ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting sympto

In [4]:
patients = data[['ชื่อ-นามสกุล (Full name)', 
                 'เพศ (Sex)', 
                 'อายุ (Age)', 
                 'เบอร์โทรศัพท์ (Telephone no.)', 
                 'Email', 
                 'ที่อยู่ปัจจุบันโปรดระบุ(Address)', 
                 'อาชีพ (Occupation)', 
                 'คณะ/สังกัด(Faculty/Department)', 
                 'อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)',
                 'เดินทางเข้าประเทศเมื่อวันที่ (Entered Thailand on) หากไม่ได้มีประวัติการเดินทางให้ลงวันที่สัมผัสใกล้ชิดผู้เข้าข่ายหรือยืนยันโรคติดเชื้อ',
                 'อาการไข้ Fever',
                 'ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting symptoms recently?)',
                 'ปัจจัยเสี่ยงอื่น']]

In [5]:
patients['ชื่อ-นามสกุล (Full name)'].fillna('', inplace=True)
patients['ชื่อ-นามสกุล (Full name)'] = patients['ชื่อ-นามสกุล (Full name)'].apply(lambda x: x.replace(u'\u200b', '').strip())
patients['คณะ/สังกัด(Faculty/Department)'].fillna('-', inplace=True)
patients['คณะ/สังกัด(Faculty/Department)'] = patients['คณะ/สังกัด(Faculty/Department)'].apply(lambda x: x.replace(u'\u200b', '').strip().upper())
patients['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)'].fillna('', inplace=True)
patients['ปัจจัยเสี่ยงอื่น'].fillna('', inplace=True)

/home/yoyo/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/yoyo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/yoyo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [6]:
def clear_place(a):
    a = a.upper()
    if 'ก้ำกึ่ง' in a or 'ก้ำกึ้ง' in a or 'ก่ำกึ่ง' in a:
        return 'ก้ำกึ่ง'
    elif 'ไม่ได้' in a or '-' in a or 'NO' in a:
        return ''
    elif 'เสี่ยง' in a or 'pui' in a:
        return 'ใกล้กลุ่มเสี่ยง'
    else:
        return a
patients['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)'] = \
        patients['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)'].apply(clear_place)

/home/yoyo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [7]:
patients.columns

Index(['ชื่อ-นามสกุล (Full name)', 'เพศ (Sex)', 'อายุ (Age)',
       'เบอร์โทรศัพท์ (Telephone no.)', 'Email',
       'ที่อยู่ปัจจุบันโปรดระบุ(Address)', 'อาชีพ (Occupation)',
       'คณะ/สังกัด(Faculty/Department)',
       'อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)',
       'เดินทางเข้าประเทศเมื่อวันที่ (Entered Thailand on) หากไม่ได้มีประวัติการเดินทางให้ลงวันที่สัมผัสใกล้ชิดผู้เข้าข่ายหรือยืนยันโรคติดเชื้อ',
       'อาการไข้ Fever',
       'ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting symptoms recently?)',
       'ปัจจัยเสี่ยงอื่น'],
      dtype='object')

In [8]:
patients['อาการไข้ Fever'].unique()

array(['ไม่มี (No)', 'มี (Yes)'], dtype=object)

In [9]:
patients['ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting symptoms recently?)'].unique()

array(['ไม่มี (No)', 'มี (Yes)', nan], dtype=object)

In [10]:
patients['ปัจจัยเสี่ยงอื่น'].unique()

array(['',
       'มีประวัติใกล้ชิด สัมผัส กับผู้ป่วยเข้าข่ายหรือยืนยันโรค COVID=19',
       'มีประวัติใกล้ชิด สัมผัส กับผู้ป่วยเข้าข่ายหรือยืนยันโรค COVID-19',
       'มีผู้อาศัยร่วมบ้านเดินทางกลับมาจากพื้นที่เสี่ยงต่อการระบาดต่อเนื่อง'],
      dtype=object)

In [11]:
patients['คณะ/สังกัด(Faculty/Department)'].unique()

array(['วารสารศาสตร์และสื่อสารมวลชน', 'แพทยศาสตร์', '-',
       'วิทยาการเรียนรู้และศึกษาศาสตร์', 'MEDICINE', 'SIIT',
       'สถาบันเทคโนโลยีนานาชาติสิรินธร', 'ศิลปศาสตร์',
       'วิทยาลัยโลกคดีศึกษา', 'สาธารณสุขศาสตร์', 'FACULTY OF MEDICINE',
       'คณะพาณิชยศาสตร์และการบัญชี', 'สำนักงานทะเบียนนักศึกษา',
       'สหเวชศาสตร์', 'พาณิชยศาสตร์และการบัญชี', 'พาณิชย์ฯ',
       'SIIT/ CIVIL ENGINEERING', 'สำนักงานทะเบียนนักศุกษา',
       'คณะพยาบาลศาสตร์', 'โครงการ BAS คณะศิลปศาสตร์',
       'โรงพยาบาลธรรมศาสตร์', 'ฝ่ายการพยาบาล รพธ.', 'สิรินธร',
       'กองบริหารงานวิชาการ', 'ทันตแพทยศาสตร์ CICM', 'สำนักงานบริหาร',
       'คณะทันตแพทยศาสตร์', 'รพธ', 'แพทย์',
       'โรงเรียนสาธิตแห่งมหาวิทยาลัยธรรมศาสตร์', 'สังคมสงเคราะห์ศาสตร์',
       'การพยาบาลผู้ป่วยโรคหัวใจและหลอดเลือด', 'คณะศิลปศาสตร์',
       'โรงเรียนอนุบาลแห่งมหาวิทยาลัยธรรมศาสตร์',
       'คณะวิทยาการเรียนรู้และศึกษาศาสตร์', 'คณะแพทยศาสตร์',
       'วิทยลัยแพทยศาสตร์นานาชาติจุฬาภรณ์',
       'วิทยาลแพทย์ศาสตรนานาชาติจุฬาภรณ์',
 

In [12]:
patients['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)'].unique()

array(['ญี่ปุ่น (JAPAN)', 'เกาหลีใต้ ( SOUTH KOREA)', 'ไต้หวัน (TAIWAN)',
       'สิงคโปร (SINGAPORE)', 'กัวลาลัมเปอร์', 'USA', 'โรงพยาบาล บีแคร์',
       'เวียดนาม',
       'สัมผัสคนไข้ COVID19:POSITIVE โดยปฏิบัติงานที่โรงพยาบาลบีแคร์',
       'MALAYSIA', 'เยอรมนี', 'TRANSIT KOREA',
       'USA (TRANSIT AT SOUTH KOREA)', 'อินโดนีเซีย',
       'อินโดนีเซีย (INDONESIA)', 'INDONESIA', 'VIETNAM',
       'อิตาลี (ITALY)', 'UK', 'BHUTAN', '', 'CAMBODIA', 'MYANMAR',
       'CAMBODIA ', 'LAOS', 'เดนมาร์ก', 'ฮ่องกง (HONG KONG)', 'VIETNAM ',
       'PHILIPPINES', 'SINGAPORE AND LAOS', ' CAME FROM USA ORIGINALLY',
       'DENMARK AND QATAR', 'DENMARK', 'TRANSIT มาเลเซีย', 'AUSTRALIA',
       'VIETNAM, CAMBODIA LAOS', 'เยอรมัน (GERMANY)',
       'TRANSFER FLIGHT ที่ JAPAN ก่อนหน้านี้นอยู่ HAWAII มา 5 สัปดาห์',
       'ฝรั่งเศส (FRANCE)', 'SAE', 'อินเดีย', 'ออสเตรีย สวิสเซอแลนด์',
       'SPAIN', 'สหราชอาณาจักร UK', 'พม่า', 'เมียนมา (พม่า)', 'ฟินแลนด์',
       'INDIA', 'FINLAND ', 'FINLAND', 'เวีย

In [13]:
patients['เดินทางเข้าประเทศเมื่อวันที่ (Entered Thailand on) หากไม่ได้มีประวัติการเดินทางให้ลงวันที่สัมผัสใกล้ชิดผู้เข้าข่ายหรือยืนยันโรคติดเชื้อ'].unique()

array(['2020-02-07T00:00:00.000000000', '2020-02-10T00:00:00.000000000',
       '2020-02-24T00:00:00.000000000', '2020-02-19T00:00:00.000000000',
       '2020-02-23T00:00:00.000000000', '2020-02-16T00:00:00.000000000',
       '2020-02-25T00:00:00.000000000', '2020-02-17T00:00:00.000000000',
       '2020-03-23T00:00:00.000000000', '2020-02-22T00:00:00.000000000',
       '2020-02-15T00:00:00.000000000', '2020-02-18T00:00:00.000000000',
       '2020-02-21T00:00:00.000000000', '2020-02-12T00:00:00.000000000',
       '2020-02-20T00:00:00.000000000', '2020-02-29T00:00:00.000000000',
       '2020-01-27T00:00:00.000000000', '2020-02-14T00:00:00.000000000',
       '2020-02-02T00:00:00.000000000', '2020-02-27T00:00:00.000000000',
       '2020-02-26T00:00:00.000000000', '2020-02-28T00:00:00.000000000',
       '2020-01-03T00:00:00.000000000', '2020-01-25T00:00:00.000000000',
       '2020-01-17T00:00:00.000000000', '2020-01-24T00:00:00.000000000',
       '2020-01-26T00:00:00.000000000', '2020-01-01

## Graph Database

In [15]:
from py2neo import Graph, Node
graph = Graph("bolt://localhost:7687", user='neo4j', password='1234')

In [16]:
graph.delete_all()

In [17]:
LEVEL_NAME = {0:'Healthy', 1:'Low Risk', 2:'High Risk', 3:'PUI'}

## Add Person Nodes

In [18]:
count = 0
for index, row in patients.iterrows():
    if row['ชื่อ-นามสกุล (Full name)']:
        
        # get patient level
        level = (row['ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting symptoms recently?)'] == 'มี (Yes)') + (len(row['ปัจจัยเสี่ยงอื่น']) > 0)
        
        if level > 1:
            print(index, level)
        
        p = Node("Person", name=row['ชื่อ-นามสกุล (Full name)'], 
                 #level=LEVEL_NAME[level],
                 sex=row['เพศ (Sex)'], 
                 age=row['อายุ (Age)'],
                 tel=row['เบอร์โทรศัพท์ (Telephone no.)'], 
                 email=row['Email'],
                 addr=row['ที่อยู่ปัจจุบันโปรดระบุ(Address)'], 
                 job=row['อาชีพ (Occupation)'],
                 dep=row['คณะ/สังกัด(Faculty/Department)'],
                 )
        p.__primarylabel__ = "Person"
        p.__primarykey__ = "name"
        p.update_labels(["Person", LEVEL_NAME[level]])
        graph.merge(p)
        count += 1
print('add ', count, ' records')

455 2
457 2
462 2
469 2
488 2
489 2
496 2
504 2
505 2
506 2
508 2
509 2
510 2
522 2
523 2
529 2
543 2
546 2
549 2
552 2
554 2
556 2
559 2
574 2
575 2
578 2
594 2
598 2
602 2
603 2
608 2
add  602  records


In [19]:
print('Total :', len(patients['ชื่อ-นามสกุล (Full name)'].unique()), 'persons (including empty string)')

Total : 485 persons (including empty string)


## Add Place Node

In [20]:
count = 0
for place in patients['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)'].unique():
    if place:
        l = Node("Location", name=place)
        l.__primarylabel__ = "Location"
        l.__primarykey__ = "name"
        graph.merge(l)
        count += 1
print('add ', count, ' places')

add  58  places


## Add Relationship

In [21]:
def get_end_of_day(d):
    return d.replace(hour=23, minute=59, second=59)

In [22]:
count = 0
for index, row in patients.iterrows():
    if row['ชื่อ-นามสกุล (Full name)']:
        # get patient level
        level = (row['ปัจจุบันยังมีอาการผิดปกติอยู่หรือไม่ (Have you been getting symptoms recently?)'] == 'มี (Yes)') + (len(row['ปัจจัยเสี่ยงอื่น']) > 0)
            
        p = Node("Person", name=row['ชื่อ-นามสกุล (Full name)'])
        p.__primarylabel__ = "Person"
        p.__primarykey__ = "name"
        p.update_labels(["Person", LEVEL_NAME[level]])
        
        
        place = row['อาศัยอยู่หรือเดินทางมาจากพื้นที่ระบาด (Resided in or returned from the area affected by the out breaks)']
        if place:
            l = Node("Location", name=place)
            l.__primarylabel__ = "Location"
            l.__primarykey__ = "name"
            
            travel_time = row['เดินทางเข้าประเทศเมื่อวันที่ (Entered Thailand on) หากไม่ได้มีประวัติการเดินทางให้ลงวันที่สัมผัสใกล้ชิดผู้เข้าข่ายหรือยืนยันโรคติดเชื้อ']
            if not pd.isnull(travel_time):
                graph.merge(Relationship( p, "WENT_TO", l, start=str(travel_time), end=str(get_end_of_day(travel_time)) ))
                count += 1

print('add ', count, ' relationships')

add  420  relationships


## Show Graph Data

In [23]:
from scripts.vis import draw, vis_network

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}
# draw(graph, options)

In [24]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='1234')

In [25]:
query = """
MATCH (p:Person)
OPTIONAL MATCH (p)-[a]->(m)
RETURN  p,
        id(p) as p_id,
        a, 
        m,
        id(m) as m_id;
"""

data = graph.run(query)
data

In [26]:
def get_node_info(node, node_id):
    gr = ''.join(str(node.labels).replace('`', '').replace(' ', '').split(':'))
    if 'Location' in gr:
        return {"id": node_id, "label": node['name'], "group": gr, "title": repr(node['name'])}
    else:
        return {"id": node_id, "label": str(node_id), "group": gr, "title": str(node_id)}

In [27]:
data = graph.run(query)

nodes = []
edges = []
physics = True

for row in data:
    p = row[0]
    p_id = row[1]
    a = row[2]
    m = row[3]
    m_id = row[4]
    
    p_info = get_node_info(p, p_id)
    
    if p_info not in nodes:
        nodes.append(p_info)
         
    if m:
        m_info = get_node_info(m, m_id)
        if m_info not in nodes:
            nodes.append(m_info)

        edges.append({"from": p_info["id"], "to": m_info["id"], "label": type(a).__name__ })
    
a = vis_network(nodes, edges, physics=physics)
a
    